### Load Data

In [1]:
import warnings
warnings.filterwarnings("ignore", message=".*'dropout_adj' is deprecated, use 'dropout_edge' instead.*")

In [2]:
import scipy.sparse as sp
import numpy as np
import json
import torch
import torch.nn.functional as F

In [3]:
# load data
adj = sp.load_npz('../CSE881_data_2024/adj.npz')
features  = np.load('../CSE881_data_2024/features.npy')
labels = np.load('../CSE881_data_2024/labels.npy')
splits = json.load(open('../CSE881_data_2024/splits.json'))
idx_train, idx_test = splits['idx_train'], splits['idx_test']

In [4]:
# transfer adjacency matrix into edge index
from torch_geometric.utils import from_scipy_sparse_matrix

edge_index = from_scipy_sparse_matrix(adj)
print("There are", edge_index[0].size(1), "edges in total in the graph\n")

print(torch.unique(edge_index[1]))
print("These edges are not weighted.")

There are 10100 edges in total in the graph

tensor([1.])
These edges are not weighted.


In [5]:
print("There are", len(features), "nodes in the graph.")
num_classes = len(np.unique(labels))
print("Each node can be one of", num_classes, "classes.")
print("Training set size:", len(idx_train))
print("Test set size:", len(idx_test))

There are 2480 nodes in the graph.
Each node can be one of 7 classes.
Training set size: 496
Test set size: 1984


In [6]:
device = 'cuda'

In [7]:
features = torch.from_numpy(features).float()
num_features = len(features[0])
print("Number of features:", num_features)

Number of features: 1390


In [8]:
features = features.to(device)
edge_index = edge_index[0].to(device)

### Supurvised Contrastive Learning Preparation

Method 'GRACE': Zhu et al., Deep Graph Contrastive Representation Learning, GRL+@ICML, 2020

https://arxiv.org/abs/2006.04131

Method 'SupCon': P. Khosla et al., Supervised Contrastive Learning, NeurIPS, 2020

https://arxiv.org/abs/2004.11362

In [9]:
import GCL.augmentors as A
import GCL.losses as L
from GCL.models import DualBranchContrast

In [10]:
class Encoder(torch.nn.Module):
    def __init__(self, encoder, augmentor):
        super(Encoder, self).__init__()
        self.encoder = encoder
        self.augmentor = augmentor

    def forward(self, x, edge_index, edge_weight=None):
        aug1, aug2 = self.augmentor
        x1, edge_index1, edge_weight1 = aug1(x, edge_index)
        x2, edge_index2, edge_weight2 = aug2(x, edge_index)
        z = self.encoder(x, edge_index)
        z1 = self.encoder(x1, edge_index1)
        z2 = self.encoder(x2, edge_index2)
        return z, z1, z2

In [11]:
def contrastive_mask(idx, labels):
    
    num_nodes = len(features)

    # create extra_pos_mask
    # initialize a 2480 x 2480 matrix of False
    extra_pos_mask = torch.zeros((num_nodes, num_nodes), dtype=torch.bool).to(device)

    # create a temporary full label tensor initialized with a dummy label and place the known labels
    full_labels = torch.full((num_nodes,), -1, dtype=labels.dtype).to(device)
    full_labels[idx] = labels

    # iterate through each known label and update the label_matrix to True by finding nodes with the same label
    for i, label in zip(idx, labels):
        same_label_indices = torch.where(full_labels == label)[0]
        extra_pos_mask[i, same_label_indices] = True
        extra_pos_mask[same_label_indices, i] = True
    extra_pos_mask.fill_diagonal_(False)

    # pos_mask: [N, 2N] for both inter-view and intra-view samples
    extra_pos_mask = torch.cat([extra_pos_mask, extra_pos_mask], dim=1).to(device)
    # fill inter-view positives only; pos_mask for intra-view samples should have False in diagonal
    extra_pos_mask.fill_diagonal_(True)

    # create extra_neg_mask
    # initialize a 2480 x 2480 matrix of True
    extra_neg_mask = torch.ones((num_nodes, num_nodes), dtype=torch.bool).to(device)

    # iterate through each known label and update the label_matrix to False by finding nodes with the same label
    for i, label in zip(idx, labels):
        same_label_indices = torch.where(full_labels == label)[0]
        extra_neg_mask[i, same_label_indices] = False
        extra_neg_mask[same_label_indices, i] = False

    # set the diagonal to False since a sample cannot be a negative of itself
    extra_neg_mask.fill_diagonal_(False)

    # neg_mask: [N, 2N] for both inter-view and intra-view samples
    extra_neg_mask = torch.cat([extra_neg_mask, extra_neg_mask], dim=1).to(device)
    
    return extra_pos_mask, extra_neg_mask

In [12]:
# augumentation of the graph
aug1 = A.Compose([A.EdgeRemoving(pe=0.1), A.FeatureDropout(pf=0.1), A.NodeDropping(pn=0.1)])
aug2 = A.Compose([A.EdgeRemoving(pe=0.1), A.FeatureDropout(pf=0.1), A.NodeDropping(pn=0.1)])

In [13]:
# contrastive loss function
contrast_loss = DualBranchContrast(loss=L.InfoNCE(tau=0.2), mode='L2L', intraview_negs=True).to(device)

### Result Predictor

In [14]:
class ResultPredictor:
    def __init__(self, model_structure, optimizer_lr, epoches, lambda_reg, output_interval=4, 
                 features=features, edge_index=edge_index, labels=labels, idx=idx_train, method='SupCon'):
        """
        Used for hyperparameter tuning with spliting 20% of training set as validation set
        
        :param model_structure: The function to define model structure
        :param optimizer_lr: The initial learning rate for the optimizer
        :param epoches: The number of epoches to train
        :param lambda_reg: Hyperparameter to control ratio of contrastive loss
        :param output_interval: The interval to ouput the training result
        :param features: The features of the dataset
        :param edge_index: The edge index tensor describing graph connectivity
        :param labels: The labels for training data in the dataset
        :param idx: Indices of the training dataset
        :param method: Contrastive learning method: SupCon or GRACE

        """
        
        self.model_structure = model_structure
        self.model = self.model_structure() 
        
        self.optimizer_lr = optimizer_lr
        self.epoches = epoches
        self.lambda_reg = lambda_reg
        
        self.output_interval = output_interval
        self.features = features
        self.edge_index = edge_index
        self.labels = labels
        self.idx = idx
        self.method = method
        
        # Converting labels to tensor
        self.labels = torch.from_numpy(self.labels).long().to(device)
        
        self.encoder_model = Encoder(encoder=self.model, augmentor=(aug1, aug2)).to(device)
        self.optimizer = torch.optim.Adam(self.encoder_model.parameters(), optimizer_lr)
        
        self.supervised_criterion = torch.nn.CrossEntropyLoss()
        self.contrast_criterion = contrast_loss
        self.extra_pos_mask, self.extra_neg_mask = contrastive_mask(self.idx, self.labels)
    
    
    def train(self):
        """
        Trains the model on the training dataset.
        """
        self.encoder_model.train()
        self.optimizer.zero_grad()
        z, z1, z2 = self.encoder_model(self.features, self.edge_index)
        
        if self.method == 'SupCon':
            contrast_loss = self.contrast_criterion(h1=z1, h2=z2, extra_pos_mask=self.extra_pos_mask, extra_neg_mask=self.extra_neg_mask)
        elif self.method == 'GRACE':
            contrast_loss = self.contrast_criterion(h1=z1, h2=z2)
        else:
            raise ValueError("Unsupported method specified. Please choose 'SupCon' or 'GRACE'.")
        supervised_loss = self.supervised_criterion(z[self.idx], self.labels)
        
        total_loss = supervised_loss + self.lambda_reg * contrast_loss
        total_loss.backward()
        self.optimizer.step()
        
        return contrast_loss, supervised_loss
    
    
    def test(self):
        """
        Evaluates the train accuracy.
        """
        self.encoder_model.eval()
        with torch.no_grad():
            out = self.encoder_model(self.features, self.edge_index)
            pred = out[0].argmax(dim=1)
            
            train_correct = pred[self.idx] == self.labels
            train_acc = int(train_correct.sum()) / len(self.idx)
            
        return train_acc
    
    
    def run(self):
        """
        Executes the training and validation process, adjusting the learning rate and lambda as needed.
        """   
        for epoch in range(self.epoches):
            contrast_loss, supervised_loss = self.train()
            train_acc = self.test()
            
            if epoch % self.output_interval == 0:
                print(f'Epoch: {epoch:03d}, Con Loss: {contrast_loss:.2f}, Sup Loss: {supervised_loss:.2f}, Train Acc: {train_acc:.2f}')
        
        return self.encoder_model
    """
    Example usage:
    tuner = HyperparameterTuner(model_structure, optimizer_lr=0.001, 
    epoches=100, lambda=0.9, output_interval=4, features=features, 
    edge_index_tensor=edge_index_tensor, labels=labels, idx=idx_train, 
    method='SupCon')
    tuner.run()
    """

### GCN

In [15]:
from torch_geometric.nn import GCN

In [16]:
def GCN_model():
    torch.manual_seed(123)
    return GCN(in_channels=num_features, hidden_channels=512, 
               out_channels=num_classes, num_layers=3, dropout=0.5, act='relu').to(device)

In [17]:
predictor = ResultPredictor(GCN_model, optimizer_lr=0.0005, epoches=210, lambda_reg=2, output_interval=20)
GCN_model = predictor.run()

Epoch: 000, Con Loss: 9.22, Sup Loss: 1.94, Train Acc: 0.15
Epoch: 020, Con Loss: 8.50, Sup Loss: 1.83, Train Acc: 0.15
Epoch: 040, Con Loss: 8.45, Sup Loss: 1.54, Train Acc: 0.57
Epoch: 060, Con Loss: 8.01, Sup Loss: 1.08, Train Acc: 0.71
Epoch: 080, Con Loss: 7.84, Sup Loss: 0.77, Train Acc: 0.74
Epoch: 100, Con Loss: 7.68, Sup Loss: 0.60, Train Acc: 0.85
Epoch: 120, Con Loss: 7.65, Sup Loss: 0.48, Train Acc: 0.86
Epoch: 140, Con Loss: 7.58, Sup Loss: 0.39, Train Acc: 0.89
Epoch: 160, Con Loss: 7.51, Sup Loss: 0.34, Train Acc: 0.92
Epoch: 180, Con Loss: 7.55, Sup Loss: 0.30, Train Acc: 0.92
Epoch: 200, Con Loss: 7.46, Sup Loss: 0.26, Train Acc: 0.93


In [18]:
GCN_model

Encoder(
  (encoder): GCN(1390, 7, num_layers=3)
)

### GraphSAGE

In [19]:
from torch_geometric.nn import GraphSAGE

In [20]:
def GraphSAGE_model():
    torch.manual_seed(123)
    return GraphSAGE(in_channels=num_features, hidden_channels=256, 
               out_channels=num_classes, num_layers=3, dropout=0.5, act='relu').to(device)

In [21]:
predictor = ResultPredictor(GraphSAGE_model, optimizer_lr=0.0003, epoches=145, lambda_reg=0.8, output_interval=20)
GraphSAGE_model = predictor.run()

Epoch: 000, Con Loss: 9.13, Sup Loss: 1.96, Train Acc: 0.05
Epoch: 020, Con Loss: 8.52, Sup Loss: 1.96, Train Acc: 0.10
Epoch: 040, Con Loss: 8.50, Sup Loss: 1.88, Train Acc: 0.27
Epoch: 060, Con Loss: 8.41, Sup Loss: 1.66, Train Acc: 0.76
Epoch: 080, Con Loss: 8.06, Sup Loss: 1.06, Train Acc: 0.89
Epoch: 100, Con Loss: 7.89, Sup Loss: 0.53, Train Acc: 0.88
Epoch: 120, Con Loss: 7.82, Sup Loss: 0.35, Train Acc: 0.92
Epoch: 140, Con Loss: 7.74, Sup Loss: 0.27, Train Acc: 0.93


In [22]:
GraphSAGE_model

Encoder(
  (encoder): GraphSAGE(1390, 7, num_layers=3)
)

### GAT

In [23]:
from torch_geometric.nn import GATConv

In [24]:
class GAT(torch.nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels, heads, dropout):
        super().__init__()
        self.gat1 = GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout) 
        self.gat2 = GATConv(hidden_channels*heads, hidden_channels, heads=heads, dropout=dropout) 
        self.gat3 = GATConv(hidden_channels*heads, out_channels, concat=False,
                             heads=1, dropout=dropout)  
    
    def forward(self, x, edge_index):
        x = self.gat1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.7, training=self.training)
        x = self.gat2(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.7, training=self.training)
        x = self.gat3(x, edge_index)
        return x

In [25]:
def GAT_model():
    torch.manual_seed(123)
    return GAT(in_channels=num_features, hidden_channels=32, 
               out_channels=num_classes, heads=16, dropout=0.7).to(device)

In [26]:
predictor = ResultPredictor(GAT_model, optimizer_lr=0.0008, epoches=67, lambda_reg=0, output_interval=8)
GAT_model = predictor.run()

Epoch: 000, Con Loss: 9.27, Sup Loss: 2.08, Train Acc: 0.35
Epoch: 008, Con Loss: 9.71, Sup Loss: 1.61, Train Acc: 0.71
Epoch: 016, Con Loss: 9.56, Sup Loss: 1.21, Train Acc: 0.83
Epoch: 024, Con Loss: 9.58, Sup Loss: 1.08, Train Acc: 0.88
Epoch: 032, Con Loss: 9.36, Sup Loss: 0.97, Train Acc: 0.91
Epoch: 040, Con Loss: 9.32, Sup Loss: 0.89, Train Acc: 0.91
Epoch: 048, Con Loss: 9.30, Sup Loss: 0.80, Train Acc: 0.92
Epoch: 056, Con Loss: 9.29, Sup Loss: 0.83, Train Acc: 0.92
Epoch: 064, Con Loss: 9.22, Sup Loss: 0.72, Train Acc: 0.94


In [27]:
GAT_model

Encoder(
  (encoder): GAT(
    (gat1): GATConv(1390, 32, heads=16)
    (gat2): GATConv(512, 32, heads=16)
    (gat3): GATConv(512, 7, heads=1)
  )
)

### GAT - v2

In [28]:
from torch_geometric.nn import GATv2Conv

In [29]:
class GATv2(torch.nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels, heads, dropout):
        super().__init__()
        self.gatv21 = GATv2Conv(num_features, hidden_channels, heads=heads, dropout=dropout)
        self.gatv22 = GATv2Conv(hidden_channels*heads, hidden_channels, heads=heads, dropout=dropout)
        self.gatv23 = GATv2Conv(hidden_channels*heads, out_channels, concat=False,
                             heads=1, dropout=dropout)  
    
    def forward(self, x, edge_index):
        x = self.gatv21(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.7, training=self.training)
        x = self.gatv22(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.7, training=self.training)
        x = self.gatv23(x, edge_index)
        return x

In [30]:
def GATv2_model():
    torch.manual_seed(123)
    return GATv2(in_channels=num_features, hidden_channels=32, 
               out_channels=num_classes, heads=16, dropout=0.7).to(device)

In [31]:
predictor = ResultPredictor(GATv2_model, optimizer_lr=0.0004, epoches=345, lambda_reg=2, output_interval=35)
GATv2_model = predictor.run()

Epoch: 000, Con Loss: 9.30, Sup Loss: 2.09, Train Acc: 0.23
Epoch: 035, Con Loss: 8.70, Sup Loss: 1.93, Train Acc: 0.45
Epoch: 070, Con Loss: 8.64, Sup Loss: 1.45, Train Acc: 0.78
Epoch: 105, Con Loss: 8.59, Sup Loss: 1.27, Train Acc: 0.86
Epoch: 140, Con Loss: 8.54, Sup Loss: 1.06, Train Acc: 0.89
Epoch: 175, Con Loss: 8.52, Sup Loss: 1.05, Train Acc: 0.90
Epoch: 210, Con Loss: 8.49, Sup Loss: 0.96, Train Acc: 0.90
Epoch: 245, Con Loss: 8.43, Sup Loss: 0.88, Train Acc: 0.91
Epoch: 280, Con Loss: 8.45, Sup Loss: 0.90, Train Acc: 0.92
Epoch: 315, Con Loss: 8.46, Sup Loss: 0.81, Train Acc: 0.93


In [32]:
GATv2_model

Encoder(
  (encoder): GATv2(
    (gatv21): GATv2Conv(1390, 32, heads=16)
    (gatv22): GATv2Conv(512, 32, heads=16)
    (gatv23): GATv2Conv(512, 7, heads=1)
  )
)

### Transformer

In [33]:
from torch_geometric.nn import TransformerConv

In [34]:
class Transformer(torch.nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels, heads, dropout):
        super().__init__()
        self.conv1 = TransformerConv(in_channels, hidden_channels, heads=heads, dropout=dropout)
        self.conv2 = TransformerConv(hidden_channels*heads, out_channels, heads=1, concat=False, dropout=dropout)
    
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.7, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [35]:
def Transformer_model():
    torch.manual_seed(123)
    return Transformer(in_channels=num_features, hidden_channels=64, 
               out_channels=num_classes, heads=8, dropout=0.7).to(device)

In [36]:
predictor = ResultPredictor(Transformer_model, optimizer_lr=0.0008, epoches=300, lambda_reg=3, output_interval=28)
Transformer_model = predictor.run()

Epoch: 000, Con Loss: 10.01, Sup Loss: 1.94, Train Acc: 0.30
Epoch: 028, Con Loss: 8.55, Sup Loss: 1.79, Train Acc: 0.30
Epoch: 056, Con Loss: 8.45, Sup Loss: 1.27, Train Acc: 0.76
Epoch: 084, Con Loss: 8.29, Sup Loss: 0.89, Train Acc: 0.86
Epoch: 112, Con Loss: 8.10, Sup Loss: 0.69, Train Acc: 0.86
Epoch: 140, Con Loss: 8.07, Sup Loss: 0.58, Train Acc: 0.87
Epoch: 168, Con Loss: 7.98, Sup Loss: 0.51, Train Acc: 0.89
Epoch: 196, Con Loss: 8.01, Sup Loss: 0.42, Train Acc: 0.92
Epoch: 224, Con Loss: 8.01, Sup Loss: 0.35, Train Acc: 0.92
Epoch: 252, Con Loss: 7.98, Sup Loss: 0.31, Train Acc: 0.94
Epoch: 280, Con Loss: 7.94, Sup Loss: 0.29, Train Acc: 0.95


In [37]:
Transformer_model

Encoder(
  (encoder): Transformer(
    (conv1): TransformerConv(1390, 64, heads=8)
    (conv2): TransformerConv(512, 7, heads=1)
  )
)

### APPNP

In [38]:
from torch_geometric.nn import APPNP
from torch.nn import Linear

In [39]:
class APPNP1(torch.nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels, K, alpha, dropout):
        super().__init__()
        self.lin1 = Linear(in_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, out_channels)
        self.appnp = APPNP(K=K, alpha=alpha, dropout=dropout)
    
    def forward(self, x, edge_index):
        x = self.lin1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.7, training=self.training)
        x = self.lin2(x)
        x = self.appnp(x, edge_index)
        return x

In [40]:
def APPNP_model():
    torch.manual_seed(123)
    return APPNP1(in_channels=num_features, hidden_channels=128, 
               out_channels=num_classes, K=2, alpha=0.1, dropout=0.7).to(device)

In [41]:
predictor = ResultPredictor(APPNP_model, optimizer_lr=0.005, epoches=360, lambda_reg=3, output_interval=28)
APPNP_model = predictor.run()

Epoch: 000, Con Loss: 9.58, Sup Loss: 1.95, Train Acc: 0.11
Epoch: 028, Con Loss: 8.50, Sup Loss: 1.83, Train Acc: 0.45
Epoch: 056, Con Loss: 8.48, Sup Loss: 1.64, Train Acc: 0.82
Epoch: 084, Con Loss: 8.44, Sup Loss: 1.40, Train Acc: 0.90
Epoch: 112, Con Loss: 8.44, Sup Loss: 1.39, Train Acc: 0.91
Epoch: 140, Con Loss: 8.44, Sup Loss: 1.39, Train Acc: 0.94
Epoch: 168, Con Loss: 8.42, Sup Loss: 1.31, Train Acc: 0.95
Epoch: 196, Con Loss: 8.41, Sup Loss: 1.32, Train Acc: 0.95
Epoch: 224, Con Loss: 8.40, Sup Loss: 1.16, Train Acc: 0.96
Epoch: 252, Con Loss: 8.42, Sup Loss: 1.25, Train Acc: 0.97
Epoch: 280, Con Loss: 8.40, Sup Loss: 1.20, Train Acc: 0.97
Epoch: 308, Con Loss: 8.40, Sup Loss: 1.13, Train Acc: 0.97
Epoch: 336, Con Loss: 8.41, Sup Loss: 1.12, Train Acc: 0.98


In [42]:
APPNP_model

Encoder(
  (encoder): APPNP1(
    (lin1): Linear(in_features=1390, out_features=128, bias=True)
    (lin2): Linear(in_features=128, out_features=7, bias=True)
    (appnp): APPNP(K=2, alpha=0.1)
  )
)

### ChebConv

In [43]:
from torch_geometric.nn import ChebConv

In [44]:
class ChebConvModel(torch.nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels, K):
        super(ChebConvModel, self).__init__()
        self.conv1 = ChebConv(in_channels, hidden_channels, K)
        self.conv2 = ChebConv(hidden_channels, out_channels, K)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.7, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [45]:
def ChebConv_model():
    torch.manual_seed(123)
    return ChebConvModel(in_channels=num_features, hidden_channels=256,
               out_channels=num_classes, K=2).to(device)

In [46]:
predictor = ResultPredictor(ChebConv_model, optimizer_lr=0.0005, epoches=230, lambda_reg=1.5, output_interval=20)
ChebConv_model = predictor.run()

Epoch: 000, Con Loss: 9.91, Sup Loss: 2.04, Train Acc: 0.22
Epoch: 020, Con Loss: 9.42, Sup Loss: 1.34, Train Acc: 0.53
Epoch: 040, Con Loss: 8.98, Sup Loss: 0.93, Train Acc: 0.81
Epoch: 060, Con Loss: 8.70, Sup Loss: 0.58, Train Acc: 0.88
Epoch: 080, Con Loss: 8.60, Sup Loss: 0.43, Train Acc: 0.91
Epoch: 100, Con Loss: 8.41, Sup Loss: 0.30, Train Acc: 0.94
Epoch: 120, Con Loss: 8.28, Sup Loss: 0.25, Train Acc: 0.96
Epoch: 140, Con Loss: 8.24, Sup Loss: 0.20, Train Acc: 0.96
Epoch: 160, Con Loss: 8.15, Sup Loss: 0.17, Train Acc: 0.97
Epoch: 180, Con Loss: 8.10, Sup Loss: 0.15, Train Acc: 0.98
Epoch: 200, Con Loss: 8.10, Sup Loss: 0.14, Train Acc: 0.98
Epoch: 220, Con Loss: 8.02, Sup Loss: 0.12, Train Acc: 0.98


In [47]:
ChebConv_model

Encoder(
  (encoder): ChebConvModel(
    (conv1): ChebConv(1390, 256, K=2, normalization=sym)
    (conv2): ChebConv(256, 7, K=2, normalization=sym)
  )
)

### Model Ensemble

In [48]:
GCN_accuracy = 0.8488
GraphSAGE_accuracy = 0.8448
GAT_accuracy = 0.8408
GATv2_accuracy = 0.8368
Transformer_accuracy = 0.8448
APPNP_accuracy = 0.8307
ChebConv_accuracy = 0.8448

In [49]:
model_group = [GCN_model, GraphSAGE_model, GAT_model, GATv2_model,
               Transformer_model, APPNP_model, ChebConv_model]
model_accuracy = [GCN_accuracy, GraphSAGE_accuracy, GAT_accuracy, GATv2_accuracy,
               Transformer_accuracy, APPNP_accuracy, ChebConv_accuracy]

In [50]:
num_nodes = len(features)
num_classes = len(np.unique(labels))

In [51]:
def weighted_voting(model_group, model_accuracy, num_nodes, num_classes, features, edge_index):
    votes = np.zeros((num_nodes, num_classes))
    group_contributions = {}
    group_names = ["GCN", "GraphSAGE", "GAT", "GATv2", "Transformer", "APPNP", "ChebConv"]

    for i, model in enumerate(model_group):
        accuracy = model_accuracy[i]
        group_name = group_names[i]

        model.eval()
        with torch.no_grad():
            out = model(features, edge_index)
            pred_probs = F.softmax(out[0], dim=1).cpu().numpy()
            weighted_pred_probs = pred_probs * accuracy
            votes += weighted_pred_probs  

        group_contributions[group_name] = np.argmax(weighted_pred_probs, axis=1)

    final_predictions = np.argmax(votes, axis=1)
    contribution_counts = {group_name: np.sum(group_contributions[group_name] == final_predictions) for group_name in group_contributions}

    return final_predictions, contribution_counts

In [52]:
final_predictions, contribution_counts = weighted_voting(model_group, model_accuracy, num_nodes, 
                                                           num_classes, features, edge_index)

In [53]:
final_predictions[idx_test][0:10]

array([1, 3, 2, 1, 1, 2, 3, 1, 1, 1], dtype=int64)

In [54]:
contribution_counts

{'GCN': 2360,
 'GraphSAGE': 2400,
 'GAT': 2356,
 'GATv2': 2374,
 'Transformer': 2368,
 'APPNP': 2367,
 'ChebConv': 2367}

In [55]:
(final_predictions[idx_train] == labels).sum().item() / len(idx_train)

0.9637096774193549

In [56]:
preds = final_predictions[idx_test]
np.savetxt('submission4.txt', preds, fmt='%d')